## 📊 実行結果の説明

### 実行順序（修正版）
1. **セル1: 環境設定とインポート確認** - パス設定、モジュールインポート
2. **セル2: BaseDemo機能テスト** - 抽象クラステスト、モデル判定、トークン計算  
3. **セル3: API呼び出しテスト** - 実際のOpenAI API呼び出し
4. **セル4: 構造解析** - ASTを使用したクラス詳細分析

### エラー修正内容
- ✅ `project_root` 未定義エラー → 各セルで再取得
- ✅ `BaseDemo` 未定義エラー → 各セルで再インポート
- ✅ `config` 未定義エラー → 各セルで再インポート
- ✅ 重複セル削除

### APIキー設定方法
```python
# 最初のセルで以下の行のコメントアウトを外す
os.environ['OPENAI_API_KEY'] = 'your-actual-api-key-here'
```

### 確認される機能
- ✅ 抽象クラス BaseDemo の正常動作  
- ✅ 設定ファイル (config.yml) の読み込み
- ✅ モデル判定機能 (reasoning vs standard)
- ✅ トークン計算・コスト算出
- ✅ OpenAI API 呼び出し（APIキー設定時）
- ✅ クラス構造詳細分析 (AST使用)

### （注） BaseDemoは抽象クラス（ABC）から継承し、抽象クラス（ABC）なので、
　　　　　　@abstractmethodで定義されたrunメソッドがあるため、
　　　　　　直接インスタンス化しようとするとTypeErrorが発生する。これは期待される動作である。

In [16]:
# TextResponseDemo の各関数詳細テスト（パス修正版）
import os
import sys
from pathlib import Path
import ast

print("\n" + "="*60)
print("🧪 TextResponseDemo 各関数の詳細テスト")
print("="*60)

print("\n📋 TextResponseDemo の構造解析:")

try:
    # パスの正確な計算
    current_dir = Path().resolve()
    project_root = current_dir.parent if current_dir.name == 'jupyter_sample' else current_dir
    sys.path.insert(0, str(project_root))
    os.chdir(str(project_root))
    
    # ファイル存在確認
    api_file_path = project_root / "a00_responses_api.py"
    print(f"🔍 ファイルパス診断:")
    print(f"   📂 現在のディレクトリ: {Path().resolve()}")
    print(f"   📂 プロジェクトルート: {project_root}")
    print(f"   📂 APIファイルパス: {api_file_path}")
    print(f"   📂 ファイル存在確認: {api_file_path.exists()}")
    
    if not api_file_path.exists():
        print("❌ a00_responses_api.py が見つかりません")
        raise SystemExit(2)
    
    # a00_responses_api.py から TextResponseDemo クラスの詳細を読み取り
    with open(api_file_path, "r", encoding="utf-8") as f:
        source = f.read()
    
    tree = ast.parse(source)
    
    # TextResponseDemo クラスを探す
    for node in ast.walk(tree):
        if isinstance(node, ast.ClassDef) and node.name == "TextResponseDemo":
            print(f"✅ TextResponseDemo クラス発見 (行番号: {node.lineno})")
            print(f"   親クラス: {[base.id for base in node.bases if hasattr(base, 'id')]}")
            
            # メソッドの詳細解析
            methods = [n for n in node.body if isinstance(n, ast.FunctionDef)]
            print(f"   メソッド数: {len(methods)}")
            
            for method in methods:
                method_name = method.name
                line_num = method.lineno
                
                # docstringの取得
                docstring = ""
                if (method.body and 
                    isinstance(method.body[0], ast.Expr) and 
                    isinstance(method.body[0].value, ast.Constant) and 
                    isinstance(method.body[0].value.value, str)):
                    docstring = method.body[0].value.value
                
                print(f"")
                print(f"   🔧 メソッド: {method_name}() (行: {line_num})")
                if docstring:
                    print(f"      説明: {docstring}")
                
                # 引数の解析
                args = []
                for arg in method.args.args:
                    if arg.arg != 'self':  # selfを除外
                        args.append(arg.arg)
                
                if args:
                    print(f"      引数: {', '.join(args)}")
                else:
                    print(f"      引数: なし（selfのみ）")
                
                # デコレーターの確認
                if method.decorator_list:
                    decorators = []
                    for dec in method.decorator_list:
                        if hasattr(dec, 'id'):
                            decorators.append(dec.id)
                        elif hasattr(dec, 'attr'):
                            decorators.append(dec.attr)
                    if decorators:
                        print(f"      デコレーター: {', '.join(decorators)}")
            
            break
    else:
        print("❌ TextResponseDemo クラスが見つかりません")
        
except Exception as e:
    print(f"❌ クラス解析エラー: {e}")
    import traceback
    traceback.print_exc()

print("\n✅ TextResponseDemo 構造解析完了")


🧪 TextResponseDemo 各関数の詳細テスト

📋 TextResponseDemo の構造解析:
🔍 ファイルパス診断:
   📂 現在のディレクトリ: /Users/nakashima_toshio/PycharmProjects/openai_api_jp
   📂 プロジェクトルート: /Users/nakashima_toshio/PycharmProjects/openai_api_jp
   📂 APIファイルパス: /Users/nakashima_toshio/PycharmProjects/openai_api_jp/a00_responses_api.py
   📂 ファイル存在確認: True
✅ TextResponseDemo クラス発見 (行番号: 253)
   親クラス: ['BaseDemo']
   メソッド数: 2

   🔧 メソッド: run() (行: 258)
      説明: デモの実行（統一化版）
      引数: なし（selfのみ）
      デコレーター: error_handler_ui, timer_ui

   🔧 メソッド: _process_query() (行: 304)
      説明: クエリの処理（統一化版）
      引数: user_input, temperature

✅ TextResponseDemo 構造解析完了


In [17]:
# TextResponseDemo の API 呼び出し機能テスト（responses.create使用・ログ抑制版）
import os
import sys
from pathlib import Path
import warnings
import logging

print("\n" + "="*60)
print("🚀 TextResponseDemo の API 呼び出し機能テスト")
print("="*60)

# プロジェクトルートをPythonパスに追加
current_dir = Path().resolve()
project_root = current_dir.parent if current_dir.name == 'jupyter_sample' else current_dir
sys.path.insert(0, str(project_root))
os.chdir(str(project_root))

# ログ出力を抑制（クリーンな表示のため）
warnings.filterwarnings('ignore')
logging.getLogger('openai_helper').setLevel(logging.ERROR)
logging.getLogger().setLevel(logging.ERROR)

# APIキーが設定されていない場合はモックテストのみ実行
if not os.getenv('OPENAI_API_KEY'):
    print("⚠️  OpenAI APIキーが未設定のため、モックテストを実行します")
    print("実際のAPIテストを行う場合は、最初のセルでAPIキーを設定してください\n")
    
    # モック版のテスト
    print("🔧 モック版 responses.create API テスト:")
    print("   - メッセージ構築: ✅ 成功（想定）")
    print("   - client.create_response()呼び出し: ⏸️  スキップ（APIキー未設定）")
    print("   - Responses API処理: ⏸️  スキップ（APIキー未設定）")
    
else:
    print("✅ OpenAI APIキーが設定済み - 実際のAPIテストを実行します\n")
    
    try:
        # 必要なインポートを再確認
        from helper_api import OpenAIClient, get_default_messages, config, ResponseProcessor
        from openai.types.responses import EasyInputMessageParam
        
        print("🔧 Responses API呼び出し準備:")
        
        # OpenAIクライアントの初期化確認
        client = OpenAIClient()
        print("   ✅ OpenAIクライアント初期化成功")
        
        # デフォルトメッセージの取得
        default_messages = get_default_messages()
        print(f"   ✅ デフォルトメッセージ取得: {len(default_messages)}件")
        
        # テスト用のユーザーメッセージ構築
        test_input = "こんにちは。簡単な挨拶をお願いします。"
        
        # メッセージリストの構築（TextResponseDemoの_process_queryと同様）
        messages = default_messages + [
            EasyInputMessageParam(role="user", content=test_input)
        ]
        print(f"   ✅ メッセージリスト構築: 合計{len(messages)}件")
        
        # API呼び出し実行
        print("\n🚀 OpenAI Responses API 呼び出し実行...")
        
        # モデル設定
        model = config.get('models.default', 'gpt-4o-mini')
        print(f"   使用モデル: {model}")
        print(f"   📡 client.create_response() 実行中...")
        
        # 実際のAPI呼び出し（responses.create使用 - 改修版）
        response = client.create_response(
            input=messages,  # responses APIでは input パラメータを使用
            model=model
        )
        
        print("   ✅ Responses API呼び出し成功!")
        
        # ResponseProcessorを使用した統一的なレスポンス処理
        formatted_response = ResponseProcessor.format_response(response)
        
        # レスポンス情報の表示
        print(f"\n📋 Responses API レスポンス詳細:")
        print(f"   ID: {formatted_response.get('id', 'N/A')}")
        print(f"   モデル: {formatted_response.get('model', 'N/A')}")
        print(f"   作成日時: {formatted_response.get('created_at', 'N/A')}")
        
        # トークン使用量
        usage = formatted_response.get('usage', {})
        if usage:
            print(f"   トークン使用量:")
            for key, value in usage.items():
                print(f"     {key}: {value}")
        
        # テキスト出力
        texts = formatted_response.get('text', [])
        if texts:
            print(f"\n🤖 Responses API応答:")
            for text in texts:
                print(f"   {text}")
        
        print(f"\n✅ TextResponseDemo Responses API機能テスト完了")
        print(f"   🔧 使用メソッド: helper_api.OpenAIClient.create_response()")
        print(f"   🔧 内部実行: self.client.responses.create(**params)")
        
    except Exception as e:
        print(f"❌ Responses API呼び出しテストエラー: {e}")
        import traceback
        traceback.print_exc()


🚀 TextResponseDemo の API 呼び出し機能テスト
✅ OpenAI APIキーが設定済み - 実際のAPIテストを実行します

🔧 Responses API呼び出し準備:
   ✅ OpenAIクライアント初期化成功
   ✅ デフォルトメッセージ取得: 3件
   ✅ メッセージリスト構築: 合計4件

🚀 OpenAI Responses API 呼び出し実行...
   使用モデル: gpt-4o-mini
   📡 client.create_response() 実行中...
   ✅ Responses API呼び出し成功!

📋 Responses API レスポンス詳細:
   ID: resp_68aefd2ecaa8819387aeaeeaabca39fd03a15ec7f0070ee0
   モデル: gpt-4o-mini-2024-07-18
   作成日時: 1756298542.0
   トークン使用量:
     input_tokens: 59
     input_tokens_details: {'cached_tokens': 0}
     output_tokens: 25
     output_tokens_details: {'reasoning_tokens': 0}
     total_tokens: 84

🤖 Responses API応答:
   こんにちは！お元気ですか？何かお手伝いできることがあれば教えてください。

✅ TextResponseDemo Responses API機能テスト完了
   🔧 使用メソッド: helper_api.OpenAIClient.create_response()
   🔧 内部実行: self.client.responses.create(**params)


In [18]:
# BaseDemo クラスの詳細診断とテスト（メソッド名修正版）
import os
import sys
from pathlib import Path

print("\n" + "="*50)
print("🔍 BaseDemo インポート・クラス詳細診断")
print("="*50)

# ステップ1: パスとディレクトリの確認・修正
current_dir = Path().resolve()
project_root = current_dir.parent if current_dir.name == 'jupyter_sample' else current_dir
sys.path.insert(0, str(project_root))

# カレントディレクトリをプロジェクトルートに変更
os.chdir(str(project_root))

print(f"1. ディレクトリ診断・修正:")
print(f"   📂 元のディレクトリ: {current_dir}")
print(f"   📂 プロジェクトルート: {project_root}")
print(f"   📂 カレントディレクトリ変更: {Path().resolve()}")
print(f"   📂 config.yml存在確認: {(project_root / 'config.yml').exists()}")
print(f"   📂 a00_responses_api.py存在確認: {(project_root / 'a00_responses_api.py').exists()}")

# ステップ2: インポート診断（Streamlit警告を無効化）
print(f"\n2. インポート詳細診断:")

# Streamlit警告を抑制
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='streamlit')

try:
    print(f"   🔧 helper_api.pyからconfig読み込み中...")
    from helper_api import config
    print(f"   ✅ config読み込み成功: {type(config)}")
    
    print(f"   🔧 a00_responses_api.pyからクラス読み込み中...")
    from a00_responses_api import BaseDemo
    print(f"   ✅ BaseDemo読み込み成功: {type(BaseDemo)}")
    print(f"   📋 BaseDemo.__bases__: {BaseDemo.__bases__}")
    print(f"   📋 BaseDemo.__module__: {BaseDemo.__module__}")
    
    from a00_responses_api import TextResponseDemo
    print(f"   ✅ TextResponseDemo読み込み成功: {type(TextResponseDemo)}")
    
except ImportError as e:
    print(f"   ❌ インポートエラー詳細: {e}")
    import traceback
    traceback.print_exc()

# ステップ3: BaseDemo抽象クラステスト（正しいエラーハンドリング）
print(f"\n3. BaseDemo 抽象クラステスト:")
print(f"   🔧 BaseDemo('test')のインスタンス化を試行...")

# インスタンス化前に抽象メソッドの確認
import abc
abstract_methods = getattr(BaseDemo, '__abstractmethods__', set())
print(f"   📋 抽象メソッド数: {len(abstract_methods)}")
print(f"   📋 抽象メソッド一覧: {list(abstract_methods)}")

# 抽象クラスのインスタンス化テスト
try:
    base_demo = BaseDemo("test")
    print(f"   ❌ 警告: BaseDemo のインスタンス化が成功しました（想定外の動作）")
    print(f"   📋 インスタンス: {type(base_demo)}")
except TypeError as e:
    print(f"   ✅ 期待された動作: BaseDemo は抽象クラスなのでインスタンス化不可")
    print(f"   📋 エラーメッセージ: {str(e)}")
except Exception as e:
    print(f"   ❌ 予期しないエラー: {type(e).__name__}: {str(e)}")

# ステップ4: TokenManager機能テスト（メソッド名修正）
print(f"\n4. TokenManager機能テスト:")
try:
    from helper_api import TokenManager
    
    test_text = "これはテスト用のテキストです。"
    tokens = TokenManager.count_tokens(test_text)
    
    # 正しいメソッド名: estimate_cost (calculate_cost ではない)
    cost = TokenManager.estimate_cost(tokens, 20, "gpt-4o")  # input_tokens, output_tokens, model
    
    print(f"   テストテキスト: '{test_text}'")
    print(f"   トークン数: {tokens}")
    print(f"   推定コスト: ${cost:.6f}")
    print(f"   ✅ 正しいメソッド名: estimate_cost(input_tokens, output_tokens, model)")
    
except Exception as e:
    print(f"   ❌ TokenManager テストエラー: {e}")

print(f"\n✅ BaseDemo 詳細診断完了")


🔍 BaseDemo インポート・クラス詳細診断
1. ディレクトリ診断・修正:
   📂 元のディレクトリ: /Users/nakashima_toshio/PycharmProjects/openai_api_jp
   📂 プロジェクトルート: /Users/nakashima_toshio/PycharmProjects/openai_api_jp
   📂 カレントディレクトリ変更: /Users/nakashima_toshio/PycharmProjects/openai_api_jp
   📂 config.yml存在確認: True
   📂 a00_responses_api.py存在確認: True

2. インポート詳細診断:
   🔧 helper_api.pyからconfig読み込み中...
   ✅ config読み込み成功: <class 'helper_api.ConfigManager'>
   🔧 a00_responses_api.pyからクラス読み込み中...
   ✅ BaseDemo読み込み成功: <class 'abc.ABCMeta'>
   📋 BaseDemo.__bases__: (<class 'abc.ABC'>,)
   📋 BaseDemo.__module__: a00_responses_api
   ✅ TextResponseDemo読み込み成功: <class 'abc.ABCMeta'>

3. BaseDemo 抽象クラステスト:
   🔧 BaseDemo('test')のインスタンス化を試行...
   📋 抽象メソッド数: 1
   📋 抽象メソッド一覧: ['run']
   ✅ 期待された動作: BaseDemo は抽象クラスなのでインスタンス化不可
   📋 エラーメッセージ: Can't instantiate abstract class BaseDemo without an implementation for abstract method 'run'

4. TokenManager機能テスト:
   テストテキスト: 'これはテスト用のテキストです。'
   トークン数: 11
   推定コスト: $0.000014
   ✅ 正しいメソッド名: estimat

In [19]:
# TextResponseDemo の機能テスト（メソッド名修正版）
import os
import sys
from pathlib import Path

print("\n" + "="*50)
print("🧪 TextResponseDemo 機能テスト")
print("="*50)

# 設定とクラスの確認
try:
    current_dir = Path().resolve()
    project_root = current_dir.parent if current_dir.name == 'jupyter_sample' else current_dir
    sys.path.insert(0, str(project_root))
    os.chdir(str(project_root))
    
    from helper_api import config, TokenManager
    from a00_responses_api import TextResponseDemo
    
    print("1. TextResponseDemo クラス詳細:")
    print(f"   📋 クラス名: {TextResponseDemo.__name__}")
    print(f"   📋 親クラス: {[base.__name__ for base in TextResponseDemo.__bases__]}")
    print(f"   📋 モジュール: {TextResponseDemo.__module__}")
    
    # TextResponseDemo の機能を部分的にテスト（Streamlit除外）
    print(f"\n2. 設定とモデル判定機能テスト:")
    
    # モデル判定機能のテスト（BaseDemoのis_reasoning_modelメソッドの模倣）
    test_models = ["gpt-4o", "o3", "gpt-5", "gpt-4o-mini"]
    reasoning_models = config.get('models.categories.reasoning', [])
    frontier_models = config.get('models.categories.frontier', [])
    
    for model in test_models:
        is_reasoning = model in reasoning_models or model in frontier_models
        print(f"   {model}: {'🧠 推論モデル' if is_reasoning else '💬 標準モデル'}")

    # トークン計算機能の確認（正しいメソッド名使用）
    print(f"\n3. TokenManager機能テスト（修正版）:")
    
    test_text = "これはTextResponseDemoのテスト用テキストです。"
    tokens = TokenManager.count_tokens(test_text)
    
    # 正しいメソッド名: estimate_cost (calculate_cost ではない)
    cost = TokenManager.estimate_cost(tokens, 20, "gpt-4o")  # input_tokens, output_tokens, model
    
    print(f"   テストテキスト: '{test_text}'")
    print(f"   入力トークン数: {tokens}")
    print(f"   出力トークン数（想定）: 20")
    print(f"   推定コスト: ${cost:.6f}")
    print(f"   ✅ 使用メソッド: TokenManager.estimate_cost()")

    # TokenManagerの利用可能メソッド一覧
    print(f"\n4. TokenManager 利用可能メソッド:")
    methods = [method for method in dir(TokenManager) if not method.startswith('_')]
    for method in methods:
        print(f"   - {method}")

    print(f"\n5. TextResponseDemo 利用可能メソッド:")
    methods = [method for method in dir(TextResponseDemo) if not method.startswith('_') or method in ['__init__']]
    print(f"   利用可能メソッド: {methods}")
    
    # 注意: Jupyter環境ではStreamlit機能は制限されるため、実際のインスタンス化はStreamlitアプリで行う
    print(f"\n   ⚠️  注意: TextResponseDemo の完全なテストはStreamlitアプリで実行してください")
    print(f"   　　（Jupyter環境ではStreamlit関連機能が制限されます）")
    
    print(f"\n✅ TextResponseDemo 機能テスト完了")

except Exception as e:
    print(f"❌ TextResponseDemo テストエラー: {e}")
    import traceback
    traceback.print_exc()


🧪 TextResponseDemo 機能テスト
1. TextResponseDemo クラス詳細:
   📋 クラス名: TextResponseDemo
   📋 親クラス: ['BaseDemo']
   📋 モジュール: a00_responses_api

2. 設定とモデル判定機能テスト:
   gpt-4o: 💬 標準モデル
   o3: 💬 標準モデル
   gpt-5: 💬 標準モデル
   gpt-4o-mini: 💬 標準モデル

3. TokenManager機能テスト（修正版）:
   テストテキスト: 'これはTextResponseDemoのテスト用テキストです。'
   入力トークン数: 14
   出力トークン数（想定）: 20
   推定コスト: $0.000014
   ✅ 使用メソッド: TokenManager.estimate_cost()

4. TokenManager 利用可能メソッド:
   - MODEL_ENCODINGS
   - count_tokens
   - estimate_cost
   - get_model_limits
   - truncate_text

5. TextResponseDemo 利用可能メソッド:
   利用可能メソッド: ['__init__', 'call_api_unified', 'create_temperature_control', 'get_model', 'handle_error', 'initialize', 'is_reasoning_model', 'run', 'show_debug_info']

   ⚠️  注意: TextResponseDemo の完全なテストはStreamlitアプリで実行してください
   　　（Jupyter環境ではStreamlit関連機能が制限されます）

✅ TextResponseDemo 機能テスト完了


In [20]:
# BaseDemo と TextResponseDemo の機能確認
import os
import sys
from pathlib import Path

# プロジェクトルートをPythonパスに追加
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

# 環境変数の確認と設定
if not os.getenv('OPENAI_API_KEY'):
    print("⚠️ OPENAI_API_KEY環境変数を設定してください")
    # os.environ['OPENAI_API_KEY'] = 'your-api-key-here'  # テスト用に設定

print(f"📂 プロジェクトルート: {project_root}")
print(f"🔧 Pythonパス: {sys.path[0]}")
print(f"🔐 OpenAI APIキー: {'✅ 設定済み' if os.getenv('OPENAI_API_KEY') else '❌ 未設定'}")

# 必要なモジュールのインポート確認
try:
    # まず基本設定をインポート
    from helper_api import ConfigManager, OpenAIClient, config, logger
    print("✅ helper_api モジュールをインポート成功")
    
    # 次にクラスをインポート
    from a00_responses_api import BaseDemo, TextResponseDemo
    print("✅ BaseDemo, TextResponseDemo をインポート成功")
    
except ImportError as e:
    print(f"❌ インポートエラー: {e}")
    import traceback
    traceback.print_exc()
    
try:
    import streamlit as st
    print("✅ streamlit インポート成功 (Jupyter環境では制限あり)")
except ImportError as e:
    print(f"❌ streamlit インポートエラー: {e}")

# グローバル変数として設定を確保
globals()['project_root'] = project_root
globals()['config'] = config

📂 プロジェクトルート: /Users/nakashima_toshio/PycharmProjects
🔧 Pythonパス: /Users/nakashima_toshio/PycharmProjects
🔐 OpenAI APIキー: ✅ 設定済み
✅ helper_api モジュールをインポート成功
✅ BaseDemo, TextResponseDemo をインポート成功
✅ streamlit インポート成功 (Jupyter環境では制限あり)
